# place cell tutorial
Singer Lab Meeting - March 1st, 2021 

In [4]:
#import libraries before we get started
import numpy as np
import pandas as pd

### Introduction

Let's say we performed an ephys recording from the hippocampus while an animal ran through a linear track. We want to identify place cells. To do this, we need to get from the raw spiking activity from all neurons to the spatial firing rate maps for spatially modulated cells. 

#insert pic here with the different steps of the analysis shown but only the start and the end visible

### Section 1 - converting raw data into data by position

Let's start with the raw(-ish) data. For each neuron, we have a vector of spike counts and positions for the whole recording. The data is structured like below:

#show image of raw spike trains x neurons 

When we look at our data in code form, it looks like this:

In [6]:
#insert matrix of spike counts and time here
#df = pd.load(toy_data) #toy data is a matrix samplesxtime,position,neuron1,neuron2,neuron3, etc.
#print(spikes)
#print(position)

First, we want to get our data put in terms of position and make a *spike counts map*. What this means is that for each neuron, we want to know how many spikes occurred in each position in the track. To do this we will count the number of spikes in each position bin. So if the track is 100m long, we will first count the spikes in the first 1-10m from the start of the track, then  11-20m from the start, then 21-30m, etc. 

In [7]:
#insert code here to convert from raw data to position data

#print(spikecounts_map)

Now we have a *spike counts map.* But that won't be enough information to get our place cells, because we also need to know how long the animal spent in each position. If the animal spent a lot of time consuming reward at the end of the track, there will be a LOT of spikes in that location, but that could be due entirely to the time spent in that location. So we now want to get an *occupancy map* to account for time.

In [8]:
#insert code here to get occupancy map

#print(occupancy_map)

### Section 2 - generating our spatial maps
We now have a spike counts map and an occupancy map. The raw data can be a bit messy across different position bins, so to clean this up we will smooth the maps. After smoothing the maps, we will combine them in order to get our final spatial map (spike counts normalized by occupancy) for each neuron.  

In [ ]:
#spikecounts_map_smooth = spikecounts_map
#occupancy_map_smooth = occupancy_map

In [10]:
#spatial_map = spikecounts_map/occupancy_map
#print(spatial_map)

### Section 3 - identifying place cells
Now we have a spatial map for every neuron, but that doesn't mean that every neuron is spatially modulated. What we want to do is identify place cells, or neurons that *are* spatially modulated. There are many different ways to do this (spatial info, reliability, thresholds, etc.), but for the purposes of this tutorial we will use some simple criteria

In [11]:
#exclusion_criteria
#inclusion_criteria
#place_cell_maps
#make figure of place cells here 

#insert picture of the full analysis process here

### Section 4 - Future directions and other notes

You now have identified place cells! You can use these place cells in other analyses, such as place cell reactivation, theta phase precession. You can also look at the spatial map features in more detail such as identifying place fields, peak firing rates, spatial info, etc. 

For real data, you will need to account for some other things such as empty position bins with zero occupancy or spatial areas with multiple dimensions. The criteria you apply to classify place/non-place cells is also important and can give you different results depending on which ones you choose to apply.